We are interested in solving systems of the form
$Xb = y$, where $X$ is a matrix of size $n \times p$ and $y$ is our $n$ observations.
Our goal here is to recover $b$ when p > n.

We will assume that $b$ is sparse meaning that $b$ has very few non-zero entries.

In [ ]:
import numpy as np
from scipy.optimize import linprog
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.stats import norm

# Question 1
Generate a dataset.
- Create a matrix $X$ of size $(n, p)$ where each entry of $X$ is sampled from $\mathcal{N}(0, 1)$
- Create a vector $b$ of size $p$ where only the $k$ first entries are non-zero
- Generate the vector $y = Xb$ of size $n$

In [ ]:


def generate_dataset(k, n, p):
    """Generate a dataset
    Generate a dataset where
    X has random entries sampled from N(0, 1),
    y such that Xb = y
    b has its k first entry equal to 1 and other entries are 0.

    Parameters
    ----------
    k : int
        Number of non-zero components of b
    n : int
        Number of samples
    p : int
        Number of features

    Returns
    -------
    X : array of size (n, p)
        Design matrix
    y : array of size (n,)
        Observations
    b : array of size (p,)
        Parameters
    """
    raise NotImplementedError


X, y, b =  generate_dataset(k=2, n=4, p=5)
np.testing.assert_allclose(X.dot(b), y)

# Question 2
## Create a solver for linear regression
Consider the problem of finding $b$ such that $Xb = y$
- If the system has no solution return None
- If the system has one or more solution, return a solution

Help: you can remove redundant equations using the SVD of $X$.
Taking $UDV$ an SVD of $X$, solving $Xb = y$ amount to solve $(DV)b = U^T y$
in particular the zeroes in $D$ should yield zeroes in $U^T y$ (If this is not the case there is no solution and
you should return None. Otherwise return a solution).
In practice you won't encounter exact zeroes so it is enough to check that $U^Ty$ has absolute value lower than 1e-10 at coordinates where $D$ has absolute value lower than 1e-10.


In [ ]:
def solve_linear_regression(X, y):
    """Find a solution of linear regression

    Find b such that Xb = y if it exists.
    Otherwise return None

    Parameters
    ----------
    X : array of size (n, p)
        Design matrix
    y : array of size (n,)
        Observations

    Return
    ------
    solution: np array of size (p,) or None
        A solution if there exists one or None if there isn't any
    """
    raise NotImplementedError

def test_solve_linear_regression():
    # An overdetermined but redondant system
    X = np.array([[1, 2], [3, 4], [2, 4], [6, 8]])
    b = np.array([1, 2])
    y = np.array([5, 11, 10, 22])
    sol = solve_linear_regression(X, y)
    np.testing.assert_allclose(b, sol)

    # An underdetermined system
    X = np.array([[2, 1, 2]])
    b = np.array([0, 1, 2])
    y = np.array([5])
    sol = solve_linear_regression(X, y)
    np.testing.assert_allclose(y, X.dot(sol))

    # A system without a solution
    X = np.array([[1, 2], [3, 4], [2, 4], [6, 8]])
    b = np.array([1, 2])
    y = np.array([5, 11, 11, 22])
    sol = solve_linear_regression(X, y)
    assert sol is None
    
test_solve_linear_regression()

# Question 3

Consider a matrix $S$ with boolean entries (True or False).

Build a solver for finding $b$ such that $Xb = y$ where $b$ has only non-zero coordinates at indexes $i$ such that $S[i] = True$ (if no such solution exists, return None)

In [ ]:

def search_in_support(S, X, y):
    """Find a solution in the support S

    Solve Xb = y with constraints b_j = 0 for all j such 
    that S[j] = False

    Parameters
    ----------
    S : boolean array of size (p,)
        Support of b
    X : array of size (n, p)
        Design matrix
    y : array of size (n,)
        Observations

    Return
    ------
    solution: np array of size (p,) or None
        A solution if there exists one or None if there isn't any
    """
    raise NotImplementedError


def test_search_in_support():
    # An overdetermined but redondant system
    X = np.array([[2, 1, 2], [4, 3, 4], [3, 2, 4], [1, 6, 8]])
    b = np.array([0, 1, 2])
    y = np.array([5, 11, 10, 22])
    sol = search_in_support(np.array([False, True, True]), X, y)
    np.testing.assert_allclose(b, sol)

    # An underdetermined system
    X = np.array([[2, 1, 2]])
    b = np.array([0, 1, 2])
    y = np.array([5])
    sol = search_in_support(np.array([True, True, True]), X, y)
    np.testing.assert_allclose(y, X.dot(sol))

    # A system without a solution
    X = np.array([[2, 1, 2], [4, 3, 4], [3, 2, 4], [1, 6, 8]])
    b = np.array([0, 1, 2])
    y = np.array([5, 11, 11, 22])
    sol = search_in_support(np.array([False, True, True]), X, y)
    assert sol is None

test_search_in_support()

# Question 4

- Build a solver that takes all possible support of a given size and tries them sequentially until it finds a solution (or nothing works).

Example: the possible supports of size $2$ in dimension $3$ are
$(True, True, False), (True, False, True), (True, True, False)$

For all supports S of size `size_support`, build a solver for finding $b$ such that $Xb = y$ where $b$ has only non-zero coordinates at indexes $i$ such that $S[i] = True$, return the first solution found. If None of the support yield a solution, return None.

- Use your solver to recover b

    * Generate a dataset of with n=10, p=20,k=4
    * Do an exhaustive search among supports of size 5
    * Check that the solution found is the correct one

In [ ]:
def exhaustive_search(size_support, X, y):
    """For all supports S of size `size_support` try to solve Xb = y
    If there exists a support that has at least one solution, return one solution.
    If None of the support yield a solution, return None

    Parameters
    ----------
    size_support: int
        The size of the support
    X : array of size (n, p)
        Design matrix
    y : array of size (n,)
        Observations

    Return
    ------
    solution: np array of size (p,) or None
        A solution if there exist one
    """
    raise NotImplementedError


def test_exhaustive_search():
    # An overdetermined but redondant system
    X = np.array([[2, 1, 2], [4, 3, 4], [3, 2, 4], [1, 6, 8]])
    b = np.array([0, 1, 2])
    y = np.array([5, 11, 10, 22])

    sol = exhaustive_search(2, X, y)
    np.testing.assert_allclose(b, sol)
    
    sol = exhaustive_search(1, X, y)
    assert sol is None


test_exhaustive_search()

In [ ]:
def find_sparse_b():
    """
    - Generate a dataset of size n=10, p=20,k=4
    - Do an exhaustive search among supports of size 5
    - Check that the solution found is the correct one
    """
    raise NotImplementedError


find_sparse_b()

# Question 5

- Use linprog to solve Basis pursuit:

$\min_b |b|$ with constraints $Xb = y$ where $| |$ is the l1 norm.

Help: BP can be recast as a linear program with n equality constraints
and 2p variables. Use variables for the positive and negative part of each entry of b.

linprog solves 

```
min_c c^T x
under the constrains 
A_eq x = b_eq
```

The solution is given by
`c_star = linprog(c=c, A_eq=A_eq, b_eq=b_eq)`

In [ ]:
def basis_pursuit(X, y):
    """ Using linprog, solve
    min_b |b| such that Xb = y where | | is the l1 norm.

    Hint: BP can be recast as a linear program with n equality constraints
    and 2p variables. Use variables for the positive and negative part
    of each entry of b

    Parameters
    -----------

    X : array of size (n, p)
        Design matrix
    y : array of size (n,)
        Observations

    Return
    -------
    sol: array of size p
        The solution of basis pursuit
    """
    raise NotImplementedError

def test_basis_pursuit():
    X = np.random.randn(10, 20)
    b = np.zeros(20)
    b[0] = 1
    b[1] = 2
    y = X.dot(b)
    np.testing.assert_array_almost_equal(b, basis_pursuit(X, y))

test_basis_pursuit()

# Question 6

Take p = 40
Take k from 1 to p
Take n from 1 to p
Generate 2 datasets with the above parameters
Solve with basis pursuit

- Build the matrix `data` of size $p, p$
where `data[i, j]` contains the number of times (0, 1 or 2) 
that basis pursuit succeeds when $k=i$ and $n=j$.

Execute `test_basis_pursuit_success` to see the phase transition


In [ ]:
def compute_basis_pursuit_success():
    """
    Take p = 40
    Take k from 1 to p
    Take n from 1 to p
    Generate 2 datasets with the above parameters
    Solve with basis pursuit
    Store the result in a matrix of size (p, p) that counts
    the number of times it succeeds for each value of n and k.

    Return
    -------
    data : np array of size (p, p)
        data[i, j] contains the number of success for n = i and k=j
    """

    raise NotImplementedError


data = compute_basis_pursuit_success()

def test_basis_pursuit_success():
    """
    Plot a heatmap of the result of basis pursuit
    and check visually that it matches the theoretical results
    """

    phi = norm.pdf
    Phi = norm.cdf

    F = lambda t: 2 * ((1 + t**2) * Phi(-t) - t * phi(t))
    delta = lambda t: 4 * phi(t) / (2 * (t * (1 - 2 * Phi(-t)) + 2 * phi(t)))
    s_max = lambda t: (delta(t) - F(t)) / ((1 + t**2) - F(t))

    grid = np.linspace(0, 10, num=500)

    plt. figure()
    plt.imshow(data, extent=[0, 1, 1, 0])
    plt.plot(s_max(grid), delta(grid))
    plt.ylabel("sample size / dimension")
    plt.xlabel("sparsity / dimension")
    plt.show()


test_basis_pursuit_success()